## Asyncio + Multiprocessing

So this is a test ... to start up a Process and within that process run an event loop
that will check status of queues and do some processing with them.

In [20]:
# killswitch=multiprocessing.Event()

class testmyasyncness(multiprocessing.Process):
    def __init__(self):
        # I have to run this?
        super(testmyasyncness, self).__init__()
        self.a=1
        self.killswitch=multiprocessing.Event()
        self.loop=asyncio.get_event_loop()
        
       
    async def print_something(self,string):
        while True:
            print(string)
            await asyncio.sleep(0.5)
        
    async def check_stop_loop(self):
        while not self.killswitch.is_set():
            await asyncio.sleep(2)
        self.loop.stop()
        print('loop stopped')
            

    async def increase_a(self):
        while True:
            self.a=self.a+1
            print(self.a)
            await asyncio.sleep(2)
            
        
    def run(self):
        self.loop.create_task(self.print_something('abc'))
        self.loop.create_task(self.check_stop_loop())
        self.loop.create_task(self.increase_a())
        self.loop.run_forever()
        
    def toggle_killswitch(self):
        # so to explain. This is a multiprocessing.Event, that is present ALSO in the separate Process!
        # since loop is running in this separate process, it doesn't know about the loop variable here
        # because this is still in the Main Process! (or an object in the main process)
        # so setting loop.stop() here would not make any sense.
        # only thing I can do is setting the killswitch here -> so it'll be passed over to the separated
        # process --> so it'll stop it in the spearate proces.
        # all process started in this cell will still have their output transferred to this output.
        self.killswitch.set()

        

In [21]:
Curator=testmyasyncness()

In [22]:
Curator.start()

abc
2
abc
abc
abc
3
abc
abc
abc
abc
4
abc
abc
abc
abc
5
abc
abc
abc
abc
loop stopped


In [23]:
Curator.toggle_killswitch()